In [1]:

import sys
import os
import random
import glob
import torch
import pandas as pd
sys.path.append("C:/Users/yuuta/Documents/fashion")
from utils.util import filter_basic_items, open_json
from utils.util import is_target_category
from utils.util import calculate_euclid_sum
# from utils.infer import id_to_vector
from utils.util import calc_roc_auc


c:\Users\yuuta\anaconda3\envs\fashion\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\yuuta\anaconda3\envs\fashion\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\yuuta\anaconda3\envs\fashion\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [2]:
import importlib
import utils
importlib.reload(utils.util)

<module 'utils.util' from 'C:\\Users/yuuta/Documents/fashion\\utils\\util.py'>

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # GPUデバイスを取得
else:
    device = torch.device("cpu")  # CPUデバイスを取得

In [4]:
negative_path = glob.glob(
    "C:/Users/yuuta/Documents/fashion/data/negative_coordinates/**.json"
    # "C:/Users/yuuta/Documents/fashion/data/test/**/*.json"
    # negative_coordinates_random
)
positive_path = glob.glob(
    "C:/Users/yuuta/Documents/fashion/data/test/**/*.json"
)

In [5]:
print(len(negative_path), len(positive_path))

30000 162975


In [6]:
id_to_vector = open_json('C:\Users\yuuta\Documents/fashion\model_learning\compatibility\data\id_to_vector.json')

In [7]:
negative_path = random.sample(negative_path, 10000)
positive_path = random.sample(positive_path, 10000)
labels = []
proposal_score = []
positive_ave = 0
negative_ave = 0

In [8]:
from utils.util import calculate_euclid_max

p_cnt = 0
for fp in positive_path:
    json_dict = open_json(fp)
    items = filter_basic_items(json_dict["items"])
    attributes = []
    vectors = []
    for item in items:
        try:
            itemId = str(item["itemId"])
        except Exception as e:
            print(fp, " : ", e)
            continue

        vector = id_to_vector[itemId]
        vectors.append(torch.tensor(vector))
    if len(vectors) != 3:
        continue
    ps = calculate_euclid_max(vectors)
    proposal_score.append(ps.to("cpu"))
    p_cnt += 1
    labels.append(0)
    positive_ave += ps
    # if p_cnt > 5000:
    #     break
positive_ave /= p_cnt
print(f"positive fin average: {positive_ave}")

C:\Users\yuuta\AppData\Local\Temp\ipykernel_15676\1282012291.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  vectors.append(torch.tensor(vector))


positive fin average: 38.85333251953125


In [9]:
n_cnt = 0
for fp in negative_path:
    json_dict = open_json(fp)
    if json_dict == None:
        continue
    parent_dir = os.path.dirname(fp)
    items = []
    attributes = []
    vectors = []
    for item in filter(is_target_category, json_dict["items"]):
        try:
            itemId = str(item["itemId"])
        except Exception as e:
            print(fp, " : ", e)
            continue

        # vectorを推定
        vector = id_to_vector[itemId]
        vectors.append(torch.tensor(vector[0]))
    if len(vectors) != 3:
        continue
    # print(len(vectors), vectors[0].shape)
    ps = calculate_euclid_max(vectors)
    proposal_score.append(ps.to("cpu"))
    labels.append(1)
    n_cnt += 1
    negative_ave += ps
    # if n_cnt > 5000:
    #     break
negative_ave /= n_cnt

print(positive_ave, negative_ave)

C:\Users\yuuta\AppData\Local\Temp\ipykernel_15676\1794572436.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  vectors.append(torch.tensor(vector[0]))


tensor(38.8533, device='cuda:0') tensor(39.3007, device='cuda:0')


In [10]:
print(len(proposal_score), len(labels))

5056 5056


In [11]:
proposal_score

[tensor(64.2583),
 tensor(20.1430),
 tensor(10.4881),
 tensor(74.7889),
 tensor(36.1861),
 tensor(37.6088),
 tensor(25.5902),
 tensor(35.4634),
 tensor(30.0535),
 tensor(57.3306),
 tensor(28.5664),
 tensor(36.3760),
 tensor(77.3723),
 tensor(96.4543),
 tensor(66.2893),
 tensor(23.6048),
 tensor(30.8350),
 tensor(81.6939),
 tensor(40.6338),
 tensor(90.5582),
 tensor(25.5707),
 tensor(16.9242),
 tensor(40.9241),
 tensor(77.9107),
 tensor(23.4074),
 tensor(14.9488),
 tensor(23.1021),
 tensor(28.0073),
 tensor(22.3482),
 tensor(33.4833),
 tensor(17.7335),
 tensor(126.2882),
 tensor(31.3085),
 tensor(21.8429),
 tensor(9.0164),
 tensor(83.4901),
 tensor(11.0166),
 tensor(29.8413),
 tensor(17.5214),
 tensor(13.7336),
 tensor(31.5156),
 tensor(17.2425),
 tensor(21.8044),
 tensor(34.2694),
 tensor(43.6071),
 tensor(21.4757),
 tensor(71.5377),
 tensor(18.1140),
 tensor(23.5754),
 tensor(94.1354),
 tensor(20.9432),
 tensor(21.2036),
 tensor(11.5409),
 tensor(37.7046),
 tensor(35.5503),
 tensor(47

In [12]:
p_auc = calc_roc_auc(labels, [p[0] for p in proposal_score], "proposal_model_image_only")
print(p_auc)

IndexError: invalid index of a 0-dim tensor. Use `tensor.item()` in Python or `tensor.item<T>()` in C++ to convert a 0-dim tensor to a number

In [ ]:
print(p_auc)

0.5551839062628233


In [ ]:
import torch

# 仮のベクトルを定義
anchor_vector = torch.tensor([1.0, 2.0, 3.0])
positive_vector = torch.tensor([2.0, 3.0, 4.0])

# ベクトルの差を計算
difference_vector = anchor_vector - positive_vector

# ノルムを計算
norm_result = torch.norm(difference_vector, dim=1)

print(norm_result.item())  # 結果を取得して表示

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)